In [ ]:
import tensorflow as tf
print(tf.__version__)
import numpy as np
import math as mt
import matplotlib.pyplot as plt

In [ ]:
StartingValuesFromFile=np.load('DiscretizedCurve10TrainingsetMultivariatePoisson.npy')
print(StartingValuesFromFile.shape)

Now calculate the outputs...

In [ ]:
YFromFile=np.load('outputsMultiVariatePoisson.npy')
print(YFromFile.shape)

In [ ]:
import os
os.getcwd()

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Lambda
from tensorflow.keras.losses import MeanSquaredError
#from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import initializers
from tensorflow.keras.layers import Input, Add
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K


inputs =  tf.keras.Input(shape=(10,))

#nodeHiddenLayer=250

#for nodeHiddenLayer in range(5,100,5):
def genNetwork(NbNodesPerLayer):

    layer = Dense(NbNodesPerLayer, activation='relu',trainable=True,
                      kernel_initializer=initializers.RandomNormal(0,1),#kernel_initializer='random_normal',
                      bias_initializer='random_normal')
    
    layer2 = Dense(NbNodesPerLayer, activation='relu',trainable=True,
                      kernel_initializer=initializers.RandomNormal(0,1),#kernel_initializer='random_normal',
                      bias_initializer='random_normal')

    outlayer=Dense(1, activation=None,trainable=True,
                      kernel_initializer=initializers.RandomNormal(0,1),#kernel_initializer='random_normal',
                      use_bias=False)
    
    tmp=layer(inputs)
    tmp2=layer2(tmp)
    finalOutput=outlayer(tmp2)
                              
    model = Model(inputs=inputs, outputs=finalOutput)

    model.compile(optimizer='adam',loss='mean_squared_error')
    return model


In [ ]:
import time
NbDataPartitions=5


errors=np.zeros(NbDataPartitions)
for k in range(10,150,10):
    ticall=time.perf_counter()
    for p in range(0,NbDataPartitions,1):
    #for p in range(0,1,1):
        StartingValues=np.concatenate((StartingValuesFromFile[:p*1000000], StartingValuesFromFile[(p+1)*1000000:]), axis=0)
        Y=np.concatenate((YFromFile[:p*1000000], YFromFile[(p+1)*1000000:]), axis=0)
        tic = time.perf_counter()
        model=genNetwork(k)
        model.compile(optimizer='adam',loss='mean_squared_error')
        model.fit(StartingValues, Y, epochs=25, batch_size=10000)
        errors[p]=model.evaluate(StartingValuesFromFile[p*1000000:(p+1)*1000000],YFromFile[p*1000000:(p+1)*1000000])
        toc = time.perf_counter()
        print(f"Calc time: {(toc - tic)/60.0:0.4f} minutes")

    filename=str('error_model_classical_2layer')+str(k)
    np.save(filename,errors)
    tocall=time.perf_counter()
print(f"Calc time: {(tocall - ticall)/60.0:0.4f} minutes")

In [ ]:
avgerror1Layer=np.zeros(14)
variance1Layer=np.zeros(14)
for k in range(1,15,1):
    error=np.load('error_model_classical_1layer'+str(k*10)+'.npy')
    avgerror1Layer[k-1]=np.average(error)
    variance1Layer[k-1]=np.var(error)
    print(k)
    print(error)


avgerror2Layer=np.zeros(14)
variance2Layer=np.zeros(14)
for k in range(1,15,1):
    error=np.load('error_model_classical_2layer'+str(k*10)+'.npy')
    avgerror2Layer[k-1]=np.average(error)
    variance2Layer[k-1]=np.var(error)
    print(k)
    print(error)
    

    


In [ ]:
print(variance1Layer)
print(variance2Layer)

In [ ]:
bestmodel1Layer=np.argmin(avgerror1Layer)
bestmodel2Layer=np.argmin(avgerror2Layer)
xvalues=range(10,150,10)

fig=plt.figure()
ax=plt.axes()
ax.set(xlabel='number of nodes in each hidden nodes', ylabel='avg mean squared error',
       title='')
ax.plot(xvalues,avgerror1Layer,markevery=[bestmodel1Layer], marker="o",label='1-layer')

ax.plot(xvalues,avgerror2Layer,markevery=[bestmodel2Layer], marker="o",label='2-layer')

plt.legend(loc='upper right')
plt.show()

In [ ]:
MCpricesTestset=np.load('outputsMultiVariatePoissonTestset100000Simulations.npy')
xvaluesTestset=np.load('DiscretizedCurve10TestsetMultivariatePoisson.npy')

In [ ]:
#retrain all with full dataset
import time

mctf=tf.reshape(tf.constant(MCpricesTestset),[MCpricesTestset.shape[0],1])
mctf=tf.dtypes.cast(mctf, tf.float32)
mse = tf.keras.losses.MeanSquaredError()
errorsModels=np.zeros(14)
for k in range(1,15,1):
    tic = time.perf_counter()
    model=genNetwork(k*10)
    model.compile(optimizer='adam',loss='mean_squared_error')
    model.fit(StartingValuesFromFile, YFromFile, epochs=25, batch_size=10000)
    predictions=model(xvaluesTestset)
    errorsModels[k-1]=mse(predictions, mctf).numpy()
    print(errorsModels[k-1])
    toc = time.perf_counter()
    print(f"Calc time: {(toc - tic)/60.0:0.4f} minutes")
    
np.save('msePointBasedMultivariate10discretized2Layer.npy',errorsModels)

In [ ]:
errorsModels1Layer=np.load('msePointBasedMultivariate10discretized.npy')
print(errorsModels1Layer)

errorsModels2Layer=np.load('msePointBasedMultivariate10discretized2Layer.npy')
print(errorsModels2Layer)

In [ ]:
bestmodel1Layer=np.argmin(errorsModels1Layer)
bestmodel2Layer=np.argmin(errorsModels2Layer)
print(errorsModels1Layer[bestmodel1Layer])
print(errorsModels2Layer[bestmodel2Layer])

xvalues=range(10,150,10)
fig=plt.figure()
ax=plt.axes()
ax.set(xlabel='number of nodes in hidden nodes', ylabel='mean squared error testset',
       title='')
ax.plot(xvalues,errorsModels1Layer,markevery=[bestmodel1Layer], marker="o",label='1-layer')
ax.plot(xvalues,errorsModels2Layer,markevery=[bestmodel2Layer], marker="o",label='2-layer')
#ax.plot(xvalues,avgerror,markevery=[bestmodel2Layer], marker="o",label='2-layer')
plt.legend(loc='upper right')
plt.ylim([0, 0.06])
plt.show()

In [ ]:
np.save('msePointBasedMultivariate10discretized',mses)